In [11]:
## setup
import numpy as np
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go

# for pretty printing dataframe that is not the last line
from IPython.display import display, HTML

pyo.init_notebook_mode(connected=True)
df = pd.read_excel("FOTP.xlsx", )
mapbox_df = pd.read_excel("Mapbox Visual for Power BI - Default Choropleth Coverage.xlsx", sheet_name="Name matches")
# format headers to be more user friendly

unformatted_header = df.columns
formatted_header = [x.lower() for x in unformatted_header]
formatted_header = [x.replace(" ", "_") for x in formatted_header]
df.columns = formatted_header

unformatted_header = mapbox_df.columns
formatted_header = [x.lower() for x in unformatted_header]
formatted_header = [x.replace(" ", "_") for x in formatted_header]
mapbox_df.columns = formatted_header

display(df.head())
display(mapbox_df.head())

# Sample code to use plotly for plotting in jupyter lab
#if FigureWidget shows error, try Figure, otherwise refer to plotly renderer to troubleshoot
# fig = go.FigureWidget(data=go.Bar(y=[2, 3, 1]))
# fig.show()

,country,edition,total_score,status,year_covered
0,Afghanistan,1994,NaN,NF,1993
1,Afghanistan,1995,90.0,NF,1994
2,Afghanistan,1996,90.0,NF,1995
3,Afghanistan,1997,100.0,NF,1996
4,Afghanistan,1998,100.0,NF,1997


,layer,matching_name,data_type
0,Global Countries,Kosovo,String / Text
1,Global Countries,Andorra,String / Text
2,Global Countries,United Arab Emirates,String / Text
3,Global Countries,Afghanistan,String / Text
4,Global Countries,Antigua and Barbuda,String / Text


In [12]:
# checking for missing values
display(df.shape)
display(df.info())
print("row with missing data:")
null_rows = df.loc[df["total_score"].isna()]
display(null_rows)
df = df.loc[df["total_score"].notna()]

(4618, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4618 entries, 0 to 4617
Data columns (total 5 columns):
country         4618 non-null object
edition         4618 non-null int64
total_score     4613 non-null float64
status          4618 non-null object
year_covered    4618 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 180.5+ KB


None

row with missing data:


,country,edition,total_score,status,year_covered
0,Afghanistan,1994,NaN,NF,1993
1930,Israeli-Occupied Territories and Palestinian A...,1994,NaN,NF,1993
3357,Rwanda,1995,NaN,NF,1994
3707,Somalia,1994,NaN,NF,1993
3708,Somalia,1995,NaN,NF,1994


In [13]:
# checking for odd data
display(df["status"].value_counts())
display(df.loc[df["status"] == "0"])
# referred back to original data which has it as 0 as well. Removing the row as its probably not rated in 2008. Freedomhouse data starts from 2009

F     1639
NF    1533
PF    1440
0        1
Name: status, dtype: int64

,country,edition,total_score,status,year_covered
2116,Kosovo,2009,0.0,0,2008


In [14]:
# checking country and date range data

print("countries in data:")
display(df["country"].nunique())
print("earliest data")
display(df["year_covered"].min())
print("latest data")
display(df["year_covered"].max())
print("country and there number of years covered")
display(df["country"].value_counts().sort_values())

countries in data:


201

earliest data


1993

latest data


2016

country and there number of years covered


Crimea                       3
Somaliland                   3
West Bank and Gaza Strip     6
South Sudan                  6
Kosovo                       9
                            ..
Belgium                     24
Spain                       24
Tonga                       24
Kiribati                    24
Equatorial Guinea           24
Name: country, Length: 201, dtype: int64

In [15]:

# check for difference in country names between fotp data and mapbox data
df["mapbox_country"] = df["country"]
country_filter = ~df["country"].isin(mapbox_df["matching_name"])
name_change_series = df.loc[country_filter, "country"]
print("names that needs to change:")
name_change_series.value_counts()


names that needs to change:


Vietnam                                                   24
The Gambia                                                24
Brunei                                                    24
Congo (Kinshasa)                                          24
Bosnia and Herzegovina                                    24
United States                                             24
Congo (Brazzaville)                                       24
Côte d'Ivoire                                             24
São Tomé and Príncipe                                     24
Saint Vincent and the Grenadines                          24
Hong Kong                                                 17
Timor-Leste                                               17
Israeli-Occupied Territories and Palestinian Authority    17
Serbia and Montenegro                                     13
West Bank and Gaza Strip                                   6
Somaliland                                                 3
Crimea                  

In [16]:
# cleaning up data
# name change for use with mapbox

name_change = {
    "mapbox_country" : {
        "Vietnam" : "Viet Nam",
        "Brunei" : "Brunei Darussalam",
        "Bosnia and Herzegovina" : "Bosnia Herzegovina",
        "United States" : "USA",
        "Congo (Brazzaville)" : "Democratic Republic of Congo",
        "Côte d'Ivoire" : "Ivory Coast",
        "Congo (Kinshasa)" : "Congo",
        "São Tomé and Príncipe" : "Sao Tome and Principe",
        "Saint Vincent and the Grenadines" : "Saint Vincent and Grenadines",
        "The Gambia" : "Gambia",
        "Timor-Leste" : "Timor Leste",
        "Israeli-Occupied Territories and Palestinian Authority" : "",
        "Serbia and Montenegro" : "",
        "West Bank and Gaza Strip" : "",
        "Somaliland" : "",
        "Crimea" : ""
    }
}
df = df.replace(name_change)

# # remove Kosovo 2008 data (year_covered)
filter1 = df["status"] == "0"
df = df.drop(index=df.index[filter1])
display(df.loc[df["country"] == "Kosovo"])
display(df.loc[df["country"] == "Congo (Brazzaville)"])
df.head()

,country,edition,total_score,status,year_covered,mapbox_country
2117,Kosovo,2010,53.0,PF,2009,Kosovo
2118,Kosovo,2011,51.0,PF,2010,Kosovo
2119,Kosovo,2012,49.0,PF,2011,Kosovo
2120,Kosovo,2013,49.0,PF,2012,Kosovo
2121,Kosovo,2014,49.0,PF,2013,Kosovo
2122,Kosovo,2015,49.0,PF,2014,Kosovo
2123,Kosovo,2016,49.0,PF,2015,Kosovo
2124,Kosovo,2017,48.0,PF,2016,Kosovo


,country,edition,total_score,status,year_covered,mapbox_country
902,Congo (Brazzaville),1994,59.0,PF,1993,Democratic Republic of Congo
903,Congo (Brazzaville),1995,53.0,PF,1994,Democratic Republic of Congo
904,Congo (Brazzaville),1996,52.0,PF,1995,Democratic Republic of Congo
905,Congo (Brazzaville),1997,45.0,PF,1996,Democratic Republic of Congo
906,Congo (Brazzaville),1998,72.0,NF,1997,Democratic Republic of Congo
907,Congo (Brazzaville),1999,75.0,NF,1998,Democratic Republic of Congo
908,Congo (Brazzaville),2000,77.0,NF,1999,Democratic Republic of Congo
909,Congo (Brazzaville),2001,71.0,NF,2000,Democratic Republic of Congo
910,Congo (Brazzaville),2002,53.0,PF,2001,Democratic Republic of Congo
911,Congo (Brazzaville),2003,55.0,PF,2002,Democratic Republic of Congo


,country,edition,total_score,status,year_covered,mapbox_country
1,Afghanistan,1995,90.0,NF,1994,Afghanistan
2,Afghanistan,1996,90.0,NF,1995,Afghanistan
3,Afghanistan,1997,100.0,NF,1996,Afghanistan
4,Afghanistan,1998,100.0,NF,1997,Afghanistan
5,Afghanistan,1999,100.0,NF,1998,Afghanistan


In [17]:
# export to csv
df.to_csv("FOTP.csv", index=False)

# Visualization (before)
![visualization before makeover](2019w2_viz_before.png)
**Note that the visualization is interactive** [Click here to see original](https://freedomhouse.org/report/freedom-world/freedom-world-2017)

# Notes
There are 5 rows with no score. Removed them from data as they have minimal impact.  
The number of countries evaluated fluctuate across the years.
Western Sahara is a disputed territory.  
Within data, the following are the same geographically:
- Israeli-Occupied Territories and Palestinian Authority (before 2012 edition)
- West Bank and Gaza Strip (starting from 2012 edition)

## Mapbox Visual limitations on Power BI
- Data shows Greenland as part of Denmark while in Mapbox, both are separate
- Greenland is shown to be under denmark as well
- Hong Kong is part of China in Mapbox while separate in data
- Data includes Somalialand which is not available in Mapbox
    - Somaliland—whose self-declared independence from Somalia is not internationally recognized
- Crimea is not available in Mapbox
- Mapbox Visual is unable to show a report page as a tooltip (This functionality is available on other map visuals in Power BI)

# What I like/dislike about the visualization

## Likes
- clicking on a country brings you to a page that provides more in-depth information

## Dislike
- the status color is not intuitive, frequent reference to the legend is required to reference press freedom status
- the legend does not give the score range that identifies if a country is Free, Partly Free or Not Free, but the score is shown when you hover over the country

# Changes made
As the title of the report is "Press Freedom's Dark Horizon", I've added a graph showing the change in NF proportion over the years to show the NF trend (A more accurate visual is the change in NF by population but there is a lack of population to show the related visual). Other change includes:

- add a legend with the score included
- press freedom distribution for the report year by country count
- change the color coding to:
    - blue for Free
    - grey for Partly Free
    - red for Not Free
    - White for NA
- clicking on a country on the map shows the score trend for the country in addition to the score for the current report

# Stretch goal
- I wanted to relate the population number with press freedom but the data is not immediately available

# Limitations of Power BI for this particular visualization
- For the country trend, I originally wanted to color code portions of the line to correspond to its freedom status but this is currently not possible in Power BI
- Changes in granularity for line graph axes is currently not possible 
- I also wanted to show country level info on hover of country as tooltip info but was not able due to limitations of Mapbox on Power BI

[Link to Power BI visual](https://app.powerbi.com/view?r=eyJrIjoiZGRhOWE2OTItNTYzMi00MjA4LWJlZGMtZTU4MGYzMWJmMjgzIiwidCI6IjIxYTg3YTllLWE0NjktNDg0ZS05ZjFmLTEwOGMxMGE1NGY3YSIsImMiOjEwfQ%3D%3D)

# Visualization (after 1)
![visualization before makeover](2019w2_viz_after_overall.png)

# Visualization (after 2)
![visualization before makeover](2019w2_viz_after_country_lvl.png)